In [138]:
# Ingest the index
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, ServiceContext, load_index_from_storage
import os

from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

import langchain 

langchain.debug = True

# from llama_index import ListIndex
# from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

In [139]:
if os.path.isfile("./storage/index_store.json") :
    print("Found index - loading - this may take a few seconds")
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    # load index
    index = load_index_from_storage(storage_context)
else :
	print("please generate an index first")

print(index)

Found index - loading - this may take a few seconds


In [140]:
tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about BioE80. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

In [141]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")

# memory = GPTIndexChatMemory(
#     index=index,
#     memory_key="chat_history",
#     query_kwargs={"response_mode": "compact"},
#     # return_source returns source nodes instead of querying index
#     return_source=True,
#     # return_messages returns context in message format
#     return_messages=True,
# )

llm = ChatOpenAI(temperature=0, max_tokens=512, model_name="gpt-3.5-turbo") # gpt-4
agent_executor = initialize_agent(tools, llm, agent="conversational-react-description", memory=memory, verbose=True)
# agent_chain = create_llama_chat_agent(tools, llm, memory=memory, verbose=True)

In [142]:
agent_executor.run(input="Hi, My name is Drew. I'm an instructor of BioE80")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Hi, My name is Drew. I'm an instructor of BioE80",
  "chat_history": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Hi, My name is Drew. I'm an instructor of BioE80",
  "chat_history": "",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant 

"Hi Drew! It's nice to meet you. How can I assist you today?"

In [143]:
cross_query_str = "Please summarize the BioE80 course"
response = agent_executor.run(input=cross_query_str)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Please summarize the BioE80 course",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: Hi Drew! It's nice to meet you. How can I assist you today?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Please summarize the BioE80 course",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: Hi Drew! It's nice to meet you. How can I assist you today?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topi

In [144]:
cross_query_str = "Will Bioe80 be offered again next year?"
response = agent_executor.run(input=cross_query_str)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Will Bioe80 be offered again next year?",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: Hi Drew! It's nice to meet you. How can I assist you today?\nHuman: Please summarize the BioE80 course\nAI: \nBioE80 is an Introduction to Bioengineering (Engineering Living Matter) course that aims to help students learn ways of thinking about engineering living matter, empower them to explore and do bioengineering starting from DNA, and become more capable of learning and explaining bioengineering to themselves and others. Additionally, the course seeks to enable students to devise and express their wishes for bioengineering as might be made true by or before 2030, and to develop practical plans for making their wishes real."
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Will Bioe80 be offered again next year?",
  "chat_histo

In [145]:
# demonstrate memory
agent_executor.run(input="What's my name?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What's my name?",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: Hi Drew! It's nice to meet you. How can I assist you today?\nHuman: Please summarize the BioE80 course\nAI: \nBioE80 is an Introduction to Bioengineering (Engineering Living Matter) course that aims to help students learn ways of thinking about engineering living matter, empower them to explore and do bioengineering starting from DNA, and become more capable of learning and explaining bioengineering to themselves and others. Additionally, the course seeks to enable students to devise and express their wishes for bioengineering as might be made true by or before 2030, and to develop practical plans for making their wishes real.\nHuman: Will Bioe80 be offered again next year?\nAI: \nIt is not possible to answer this question with the given context information."
}
[chain/start] [1:chain:AgentExecutor > 2:chain:

'\nDrew Endy is a dear friend of Ahmed Best and a professor of bioengineering. He is known for his creative and accessible approach to bioengineering, which does not frighten people but instead excites them. Ahmed Best is working with Drew to amplify this message and to get more people to tell stories about optimistic futures.'

In [146]:
# # reinitialize agent
# memory = ConversationBufferMemory(memory_key="chat_history")
# llm = OpenAI(temperature=0)
# agent_chain = create_llama_chat_agent(
#     toolkit,
#     llm,
#     memory=memory,
# )

In [ ]:
while True:
    text_input = input("User: ")
    response = agent_executor.run(input=text_input)
    print(f"Agent: {response}")

User:  What does "abstraction" refer to in the context of BioE80?


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What does \"abstraction\" refer to in the context of BioE80?",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: Hi Drew! It's nice to meet you. How can I assist you today?\nHuman: Please summarize the BioE80 course\nAI: \nBioE80 is an Introduction to Bioengineering (Engineering Living Matter) course that aims to help students learn ways of thinking about engineering living matter, empower them to explore and do bioengineering starting from DNA, and become more capable of learning and explaining bioengineering to themselves and others. Additionally, the course seeks to enable students to devise and express their wishes for bioengineering as might be made true by or before 2030, and to develop practical plans for making their wishes real.\nHuman: Will Bioe80 be offered again next year?\nAI: \nIt is not possible to answer this question with the given context information.\nHuma

User:  Please explain how to make artemisinin


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Please explain how to make artemisinin",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: Hi Drew! It's nice to meet you. How can I assist you today?\nHuman: Please summarize the BioE80 course\nAI: \nBioE80 is an Introduction to Bioengineering (Engineering Living Matter) course that aims to help students learn ways of thinking about engineering living matter, empower them to explore and do bioengineering starting from DNA, and become more capable of learning and explaining bioengineering to themselves and others. Additionally, the course seeks to enable students to devise and express their wishes for bioengineering as might be made true by or before 2030, and to develop practical plans for making their wishes real.\nHuman: Will Bioe80 be offered again next year?\nAI: \nIt is not possible to answer this question with the given context information.\nHuman: What's my name?\nAI